In [1]:
pip install git+https://github.com/HeliXonProtein/OmegaFold.git


  Cloning https://github.com/HeliXonProtein/OmegaFold.git to /tmp/pip-req-build-45x06p5t
  Running command git clone --filter=blob:none --quiet https://github.com/HeliXonProtein/OmegaFold.git /tmp/pip-req-build-45x06p5t
  Resolved https://github.com/HeliXonProtein/OmegaFold.git to commit 313c873ad190b64506a497c926649e15fcd88fcd
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 GB 707.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 11.9 MB/s eta 0:00:00
  Created wheel for OmegaFold: filename=OmegaFold-0.0.0-py3-none-any.whl size=55664 sha256=20e996e5ac6616bf4dd48a5f9fdeb23260c8ca381ccf891f37828d9506b40d15
  Stored in directory: /tmp/pip-ephem-wheel-cache-gfxt2mk1/wheels/fe/57/01/3ce12996dd37debe3ee6a02e8748fffdd4df7b885c5bb8071d
Successfully built OmegaFold
  Attempting uninstall: torch
    Found existing installation: torch 2.2.1+cu121
    Uninstalling torch-2.2.1+cu121:
      Successfully uninstalled torch-2.2.1

In [2]:
#do a fake run of omegafold before mounting to drive to download the weights etc.
!omegafold /thisdoesnotexits.fa /content/output

INFO:root:Downloading weights from https://helixon.s3.amazonaws.com/release1.pt to /root/.cache/omegafold_ckpt/model.pt
100% 2.96G/2.96G [01:45<00:00, 30.0MB/s]
INFO:root:Constructing OmegaFold
INFO:root:Reading /thisdoesnotexits.fa
Traceback (most recent call last):
  File "/usr/local/bin/omegafold", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.10/dist-packages/torch/autograd/grad_mode.py", line 27, in decorate_context
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/omegafold/__main__.py", line 58, in main
    for i, (input_data, save_path) in enumerate(
  File "/usr/local/lib/python3.10/dist-packages/omegafold/pipeline.py", line 120, in fasta2inputs
    with open(fasta_path, 'r') as file:
FileNotFoundError: [Errno 2] No such file or directory: '/thisdoesnotexits.fa'


In [3]:
from google.colab import drive
drive.mount('/content/drive')

import os
import shutil
import glob
import json
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import uuid
from datetime import datetime
import re
import torch
import time

meta_data_filepath = "/content/drive/MyDrive/Generative_Models/IL10_scaffolding/metadata_omegafold.csv"

if os.path.exists(meta_data_filepath):
  all_metadata_df = pd.read_csv(meta_data_filepath)
  print("Existing generation metadata read in.")
else:
  all_metadata_df = pd.DataFrame()
  #all_metadata_df.to_csv(meta_data_filepath, index=False)
  print("Created omegafold metadata dataframe")

Mounted at /content/drive
Existing generation metadata read in.


In [4]:
def count_records_fasta(fasta_file):
  num_records = 0
  with open(fasta_file, 'r') as f:
    for line in f:
      if line.startswith('>'):
        num_records += 1
  return num_records


In [ ]:
list(all_metadata_df.loc[:,'input_file_path'])

In [ ]:
import os

for filename in os.listdir("/content/drive/MyDrive/Generative_Models/IL10_scaffolding/rfdiffusion_il10/MPNN_redesigns/"):
  if filename.endswith(".fa"):

    if not ("tev" in filename): continue
    input_file_path = "/content/drive/MyDrive/Generative_Models/IL10_scaffolding/rfdiffusion_il10/MPNN_redesigns/" + filename
    print(input_file_path)
    if input_file_path in list(all_metadata_df.loc[:,'input_file_path']):
      print('already processed...')
      continue


In [5]:
import os
import os

for filename in os.listdir("/content/drive/MyDrive/Generative_Models/IL10_scaffolding/rfdiffusion_il10/MPNN_redesigns/"):
  if filename.endswith(".fa"):

    if not ("tev" in filename): continue
    input_file_path = "/content/drive/MyDrive/Generative_Models/IL10_scaffolding/rfdiffusion_il10/MPNN_redesigns/" + filename
    print(input_file_path)
    if input_file_path in list(all_metadata_df.loc[:,'input_file_path']):
      print('already processed...')
      continue
    #input_file_path = "/content/drive/MyDrive/Generative_Models/IL10_scaffolding/refold.fa"
    #input_file_path = "/content/drive/MyDrive/Generative_Models/IL10_scaffolding/rfdiffusion_il10/mpnn_redesigns"

    output_dir_path = "/content/drive/MyDrive/Generative_Models/IL10_scaffolding/rfdiffusion_il10/folded_structures"

    meta_data_entry = pd.Series()
    meta_data_entry["model"] = "omegafold"
    meta_data_entry["task"] = "Structure Prediction"
    meta_data_entry["input_file_path"] = input_file_path
    meta_data_entry["output_dir_path"] = output_dir_path
    meta_data_entry["Timestamp"] = str(datetime.now())
    meta_data_entry['gpu'] = 'T4 GPU'
    meta_data_entry["batch_id"] = str(uuid.uuid4())
    meta_data_entry['batch_size'] = count_records_fasta(input_file_path)

    start_time = time.time()

    !omegafold {input_file_path} {output_dir_path}

    end_time = time.time()
    total_job_time = end_time - start_time
    meta_data_entry['wall_time_batch'] = str(total_job_time) + " Seconds"
    meta_data_entry['wall_time_task'] = str(total_job_time/meta_data_entry['batch_size']) + " Seconds (inferred)"
    all_metadata_df = pd.concat([all_metadata_df,pd.DataFrame(meta_data_entry).T], ignore_index=True)
    all_metadata_df.to_csv(meta_data_filepath, index=False)


/content/drive/MyDrive/Generative_Models/IL10_scaffolding/rfdiffusion_il10/MPNN_redesigns/rfdiffusion_tev_mono_8d72f918-8456-442f-80c4-d4bb75e2f83d.fa
already processed...
/content/drive/MyDrive/Generative_Models/IL10_scaffolding/rfdiffusion_il10/MPNN_redesigns/rfdiffusion_tev_mono_b21d2247-fa03-4f61-9a9d-e60de42058e8.fa
already processed...
/content/drive/MyDrive/Generative_Models/IL10_scaffolding/rfdiffusion_il10/MPNN_redesigns/rfdiffusion_tev_mono_8d69b48f-4c4a-40c7-9f77-a2292ed117c1.fa
already processed...
/content/drive/MyDrive/Generative_Models/IL10_scaffolding/rfdiffusion_il10/MPNN_redesigns/rfdiffusion_tev_mono_da316ead-f92c-436d-ba1e-3c8df4dc194b.fa
already processed...
/content/drive/MyDrive/Generative_Models/IL10_scaffolding/rfdiffusion_il10/MPNN_redesigns/rfdiffusion_tev_mono_7fe54120-6687-4dc8-9d09-132f94e32207.fa
already processed...
/content/drive/MyDrive/Generative_Models/IL10_scaffolding/rfdiffusion_il10/MPNN_redesigns/rfdiffusion_tev_mono_82025020-563a-4432-bbf0-56a52

In [ ]:
import os

for filename in os.listdir("/content/drive/MyDrive/Generative_Models/IL10_scaffolding/rfdiffusion_il10/MPNN_redesigns/"):
  if filename.endswith(".fa"):
    input_file_path = "/content/drive/MyDrive/Generative_Models/IL10_scaffolding/rfdiffusion_il10/MPNN_redesigns/" + filename

    if not ("bcc75b66-99b3-48c9-82b7-139757befa83"in input_file_path): continue
    output_dir_path = "/content/drive/MyDrive/Generative_Models/IL10_scaffolding/rfdiffusion_il10/folded_structures"

    meta_data_entry = pd.Series()
    meta_data_entry["model"] = "omegafold"
    meta_data_entry["task"] = "Structure Prediction"
    meta_data_entry["input_file_path"] = input_file_path
    meta_data_entry["output_dir_path"] = output_dir_path
    meta_data_entry["Timestamp"] = str(datetime.now())
    meta_data_entry['gpu'] = 'T4 GPU'
    meta_data_entry["batch_id"] = str(uuid.uuid4())
    meta_data_entry['batch_size'] = count_records_fasta(input_file_path)

    start_time = time.time()

    !omegafold {input_file_path} {output_dir_path}

    end_time = time.time()
    total_job_time = end_time - start_time
    meta_data_entry['wall_time_batch'] = str(total_job_time) + " Seconds"
    meta_data_entry['wall_time_task'] = str(total_job_time/meta_data_entry['batch_size']) + " Seconds (inferred)"
    all_metadata_df = pd.concat([all_metadata_df,pd.DataFrame(meta_data_entry).T], ignore_index=True)
    all_metadata_df.to_csv(meta_data_filepath, index=False)


INFO:root:Loading weights from /root/.cache/omegafold_ckpt/model.pt
INFO:root:Constructing OmegaFold
INFO:root:Reading /content/drive/MyDrive/Generative_Models/IL10_scaffolding/rfdiffusion_il10/MPNN_redesigns/rfdiffusion_il10_mono_bcc75b66-99b3-48c9-82b7-139757befa83.fa
INFO:root:Predicting 1th chain in /content/drive/MyDrive/Generative_Models/IL10_scaffolding/rfdiffusion_il10/MPNN_redesigns/rfdiffusion_il10_mono_bcc75b66-99b3-48c9-82b7-139757befa83.fa
INFO:root:184 residues in this chain.
INFO:root:Finished prediction in 56.83 seconds.
INFO:root:Saving prediction to /content/drive/MyDrive/Generative_Models/IL10_scaffolding/rfdiffusion_il10/folded_structures/RFdiffusion_150it_il10_bcc75b66-99b3-48c9-82b7-139757befa83_design1.pdb
INFO:root:Saved
INFO:root:Predicting 2th chain in /content/drive/MyDrive/Generative_Models/IL10_scaffolding/rfdiffusion_il10/MPNN_redesigns/rfdiffusion_il10_mono_bcc75b66-99b3-48c9-82b7-139757befa83.fa
INFO:root:184 residues in this chain.
INFO:root:Finished pr

In [ ]:
import os

for filename in os.listdir("/content/drive/MyDrive/Generative_Models/IL10_scaffolding/rfdiffusion_il10/MPNN_redesigns/"):
  if filename.endswith(".fa"):
    input_file_path = "/content/drive/MyDrive/Generative_Models/IL10_scaffolding/rfdiffusion_il10/MPNN_redesigns/" + filename

    if input_file_path in list(all_metadata_df.loc[:,'input_file_path']): continue
    output_dir_path = "/content/drive/MyDrive/Generative_Models/IL10_scaffolding/rfdiffusion_il10/folded_structures"

    meta_data_entry = pd.Series()
    meta_data_entry["model"] = "omegafold"
    meta_data_entry["task"] = "Structure Prediction"
    meta_data_entry["input_file_path"] = input_file_path
    meta_data_entry["output_dir_path"] = output_dir_path
    meta_data_entry["Timestamp"] = str(datetime.now())
    meta_data_entry['gpu'] = 'T4 GPU'
    meta_data_entry["batch_id"] = str(uuid.uuid4())
    meta_data_entry['batch_size'] = count_records_fasta(input_file_path)

    start_time = time.time()

    !omegafold {input_file_path} {output_dir_path}

    end_time = time.time()
    total_job_time = end_time - start_time
    meta_data_entry['wall_time_batch'] = str(total_job_time) + " Seconds"
    meta_data_entry['wall_time_task'] = str(total_job_time/meta_data_entry['batch_size']) + " Seconds (inferred)"
    all_metadata_df = pd.concat([all_metadata_df,pd.DataFrame(meta_data_entry).T], ignore_index=True)
    all_metadata_df.to_csv(meta_data_filepath, index=False)


In [ ]:
all_metadata_df

,model,task,input_file_path,output_dir_path,Timestamp,gpu,batch_id,batch_size,wall_time_batch,wall_time_task
0,omegafold,Structure Prediction,/content/drive/MyDrive/Generative_Models/IL10_...,/content/drive/MyDrive/Generative_Models/IL10_...,2024-05-15 23:34:02.491410,T4 GPU,154ddb1d-a61a-4712-b365-a78947b5d94b,102,6475.831564426422 Seconds,63.488544749278645 Seconds (inferred)


In [ ]:
!ls /content/drive/MyDrive/Generative_Models/Utilities/MPNN_backbone/len100/merged.fa

foldingdiff  framediff	genie_swissprot_l_256  merged.fa  proteinsgm  RFdiffusion_150it


In [ ]:
meta_data_entry

model                                                      omegafold
task                                            Structure Prediction
input_file_path    /content/drive/MyDrive/Generative_Models/Utili...
output_dir_path    /content/drive/MyDrive/Generative_Models/Utili...
Timestamp                                 2024-04-04 22:44:53.835326
gpu                                                           T4 GPU
batch_id                        ddfe6bd0-8d52-4c84-8bb9-049d8574a3c8
batch_size                                                       735
wall_time_batch                            14482.58325266838 Seconds
wall_time_task                 19.704194901589634 Seconds (inferred)
dtype: object

In [ ]:
# prompt: kill the colab runtime

!kill -9 -1
